In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import coiled
import dask
import xarray as xr
import xclim

from ocr.risks.fire import classify_wind_directions, direction_histogram, nws_fire_weather
from ocr.utils import load_conus404

In [3]:
args = {
    'name': 'fire-weather-distribution',
    'region': 'us-west-2',
    'n_workers': [20, 20],
    'tags': {'Project': 'OCR'},
    'worker_vm_types': 'm8g.8xlarge',
    'scheduler_vm_types': 'm8g.2xlarge',
}
cluster = coiled.Cluster(**args)
client = cluster.get_client()

Output()

╭──────────────────────────────── Package Info ────────────────────────────────╮
│                ╷                                                             │
│   Package      │ Note                                                        │
│ ╶──────────────┼───────────────────────────────────────────────────────────╴ │
│   ocr          │ Wheel built from ~/devel/carbonplan/ocr                     │
│                ╵                                                             │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── Not Synced with Cluster ───────────────────────────╮
│           ╷                                                      ╷           │
│   Package │ Error                                                │ Level     │
│ ╶─────────┼──────────────────────────────────────────────────────┼─────────╴ │
│   s5cmd   │ s5cmd~=2.3.0 has no install candidate for Python     │ Warning   │
│           │ 3.13 linux-aarch64 on conda-forge                    │           │
│           ╵                                                      ╵           │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

2025-08-06 15:33:56,387 - distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=20 maximum=20


In [4]:
# Make relative humidity intermediate data variable

In [5]:
ds = load_conus404()
ds

<xarray.Dataset> Size: 13TB
Dimensions:  (time: 376945, y: 1015, x: 1367)
Coordinates:
  * y        (y) float64 8kB -2.028e+06 -2.024e+06 ... 2.024e+06 2.028e+06
  * x        (x) float64 11kB -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * time     (time) datetime64[ns] 3MB 1979-10-01 ... 2022-10-01
Data variables:
    PSFC     (time, y, x) float32 2TB dask.array<chunksize=(376945, 10, 10), meta=np.ndarray>
    Q2       (time, y, x) float32 2TB dask.array<chunksize=(376945, 10, 10), meta=np.ndarray>
    T2       (time, y, x) float32 2TB dask.array<chunksize=(376945, 10, 10), meta=np.ndarray>
    TD2      (time, y, x) float32 2TB dask.array<chunksize=(376945, 10, 10), meta=np.ndarray>
    U10      (time, y, x) float32 2TB dask.array<chunksize=(376945, 10, 10), meta=np.ndarray>
    V10      (time, y, x) float32 2TB dask.array<chunksize=(376945, 10, 10), meta=np.ndarray>
Attributes: (12/148)
    AER_ANGEXP_OPT:                  1
    AER_ANGEXP_VAL:                  1.2999999523162842
    AER_AOD550_OPT:                  1
    AER_AOD550_VAL:                  0.11999999731779099
    AER_ASY_OPT:                     1
    AER_ASY_VAL:                     0.8999999761581421
    ...                              ...
    WEST-EAST_PATCH_START_STAG:      1
    WEST-EAST_PATCH_START_UNSTAG:    1
    W_DAMPING:                       1
    YSU_TOPDOWN_PBLMIX:              0
    history:                         Tue Mar 29 16:35:22 2022: ncrcat -A -vW ...
    history_of_appended_files:       Tue Mar 29 16:35:22 2022: Appended file ...

In [6]:
%%time
recreate_rh = False
if recreate_rh:
    hurs = xclim.indicators.atmos.relative_humidity_from_dewpoint(tas=ds['T2'], tdps=ds['TD2'])
    hurs = dask.optimize(hurs)[0]
    hurs.to_zarr(
        's3://carbonplan-ocr/intermediate/met-data/conus404/hurs.zarr',
        consolidated=True,
        mode='w',
    )

CPU times: user 2 μs, sys: 6 μs, total: 8 μs
Wall time: 9.78 μs


In [7]:
# Make sfcWindspeed and sfcWindfromdir intermediate variables

In [8]:
%%time
recreate_winds = False
if recreate_winds:
    winds = xclim.indicators.atmos.wind_speed_from_vector(uas=ds['U10'], vas=ds['V10'])[
        0
    ].to_dataset()
    winds['sfcWindfromdir'] = xclim.indicators.atmos.wind_speed_from_vector(
        uas=ds['U10'], vas=ds['V10']
    )[1]
    winds = dask.optimize(winds)[0]
    winds.to_zarr(
        's3://carbonplan-ocr/intermediate/met-data/conus404/winds.zarr',
        consolidated=True,
        mode='w',
    )

CPU times: user 6 μs, sys: 1e+03 ns, total: 7 μs
Wall time: 9.78 μs


In [9]:
# calculate whether there is fire weather

In [10]:
hurs = xr.open_zarr('s3://carbonplan-ocr/intermediate/met-data/conus404/hurs.zarr')
wind = xr.open_zarr('s3://carbonplan-ocr/intermediate/met-data/conus404/winds.zarr')
fire_weather_mask = nws_fire_weather(
    hurs['hurs'],
    15,
    # reason that wind gusts are typically ~40% higher than average wind speed
    # and we want to base this on wind gusts (need a citation for this)
    wind['sfcWind'] * 1.4,
    35,
)

fire_weather_mask = dask.optimize(fire_weather_mask)[0]

In [11]:
%%time
recreate_direction_indices = False
if recreate_direction_indices:
    direction_indices = classify_wind_directions(wind['sfcWindfromdir'])
    direction_indices.to_zarr(
        's3://carbonplan-ocr/intermediate/met-data/conus404/winds-direction-indices.zarr',
        consolidated=True,
        mode='w',
    )

direction_indices = xr.open_zarr(
    's3://carbonplan-ocr/intermediate/met-data/conus404/winds-direction-indices.zarr'
)['wind_direction_classification']

direction_indices

CPU times: user 40.5 ms, sys: 5.8 ms, total: 46.3 ms
Wall time: 513 ms


<xarray.DataArray 'wind_direction_classification' (time: 376945, y: 1015,
                                                   x: 1367)> Size: 2TB
dask.array<open_dataset-wind_direction_classification, shape=(376945, 1015, 1367), dtype=float32, chunksize=(376945, 10, 10), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-10-01 ... 2022-10-01
  * x        (x) float64 11kB -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * y        (y) float64 8kB -2.028e+06 -2.024e+06 ... 2.024e+06 2.028e+06
Attributes:
    long_name:         wind direction classified into 8 cardinal directions (...
    short_name:        wind_direction_classification
    direction_labels:  ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']

In [12]:
%%time

# we are going to reuse this. we should save it to a zarr store
# then we can apply the fire_weather_mask dynamically.
masked_direction_indices = direction_indices.where(fire_weather_mask)

# create the distribution of winds based upon the classified wind directions
# if there are nans in the mix, they won't contribute to the distribution
fraction = direction_histogram(masked_direction_indices)
fraction = dask.optimize(fraction)[0]
fraction

CPU times: user 792 ms, sys: 13.1 ms, total: 805 ms
Wall time: 808 ms


<xarray.DataArray 'wind_direction_histogram' (direction: 8)> Size: 32B
dask.array<where, shape=(8,), dtype=float32, chunksize=(8,), chunktype=numpy.ndarray>
Coordinates:
  * direction  (direction) float32 32B 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0
Attributes:
    long_name:  Wind Direction Histogram
    units:      probability

In [ ]:
%%time
fraction.to_zarr(
    's3://carbonplan-ocr/intermediate/met-data/conus404/fire_weather_wind_distribution-hurs15_wind35.zarr',
    mode='w',
    consolidated=True,
)

/Users/andersy005/devel/carbonplan/ocr/.pixi/envs/default/lib/python3.13/site-packages/zarr/api/asynchronous.py:229: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [ ]:
fraction

In [ ]:
%%time
fraction = xr.open_zarr(
    's3://carbonplan-ocr/intermediate/met-data/conus404/fire_weather_wind_distribution-hurs15_wind35.zarr'
)['wind_direction_histogram']
# sum up the fractions of all of the wind directions. if any pixel sums to zero it will
# fall into the no_fire_weather mask and be cast to a `nan`. if any pixel has any fraction
# of timestamps with fire weather then it will be cast into the mask where there *is* fire weather
no_fire_weather = fraction.sum(dim='wind_direction')
mode = fraction.argmax(dim='wind_direction').where(no_fire_weather).chunk({'x': -1, 'y': -1})


mode.to_zarr(
    's3://carbonplan-ocr/intermediate/met-data/conus404/fire_weather_wind_mode-hurs15_wind35.zarr',
    mode='w',
    consolidated=True,
)

In [ ]:
test = xr.open_zarr(
    's3://carbonplan-ocr/intermediate/met-data/conus404/fire_weather_wind_mode-hurs15_wind35.zarr'
)

In [ ]:
test

In [ ]:
cluster.close()